# Submissions are evaluated using **area under the ROC curve** using the predicted probabilities and the ground truth targets.

In [1]:
!pip install lightgbm

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, \
accuracy_score,f1_score, mean_absolute_error, mean_squared_error, r2_score
# all classifcation models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, \
GradientBoostingClassifier, RandomForestClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth',400)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv',index_col = 'id')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv',index_col = 'id')
submission_data = pd.read_csv('/kaggle/input/playground-series-s4e7/sample_submission.csv')
print(len(train_data), len(test_data), len(submission_data))

11504798 7669866 7669866


In [4]:
train_data.describe()

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,11504798.00000,11504798.00000,11504798.00000,11504798.00000,11504798.00000,11504798.00000,11504798.00000,11504798.00000
mean,38.38356,0.99802,26.41869,0.46300,30461.37041,112.42544,163.89774,0.12300
std,14.99346,0.04443,12.99159,0.49863,16454.74521,54.03571,79.97953,0.32843
min,20.00000,0.00000,0.00000,0.00000,2630.00000,1.00000,10.00000,0.00000
25%,24.00000,1.00000,15.00000,0.00000,25277.00000,29.00000,99.00000,0.00000
50%,36.00000,1.00000,28.00000,0.00000,31824.00000,151.00000,166.00000,0.00000
75%,49.00000,1.00000,35.00000,1.00000,39451.00000,152.00000,232.00000,0.00000
max,85.00000,1.00000,52.00000,1.00000,540165.00000,163.00000,299.00000,1.00000


In [7]:
print(train_data.shape)
train_data.drop_duplicates(inplace=True)
print(train_data.shape)

(11504798, 11)
(11504798, 11)


In [8]:
train_data[:5]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
0,Male,21,1,35.00000,0,1-2 Year,Yes,65101.00000,124.00000,187,0
1,Male,43,1,28.00000,0,> 2 Years,Yes,58911.00000,26.00000,288,1
2,Female,25,1,14.00000,1,< 1 Year,No,38043.00000,152.00000,254,0
3,Female,35,1,1.00000,0,1-2 Year,Yes,2630.00000,156.00000,76,0
4,Female,36,1,15.00000,1,1-2 Year,No,31951.00000,152.00000,294,0


In [19]:
pd.set_option('display.max_columns',500)
train_data['Annual_Premium'].unique()

array([65101., 58911., 38043., ..., 22112., 54335., 64538.])